In [3]:
# 📚 Imports
from datasets import load_dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import pandas as pd

# 🔄 Load both splits
dataset = load_dataset("Suvo696/InSDNN")
df_train = dataset["train"].to_pandas()
df_test = dataset["test"].to_pandas()

print("Train shape:", df_train.shape)
print("Test shape:", df_test.shape)


Train shape: (275110, 84)
Test shape: (68778, 84)


In [6]:
df_train

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,162175,200.175.2.130,43226,192.168.3.130,80,6,9/1/2020 16:30,15163,4,4,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,PROBE
1,62805,192.168.20.133,51325,192.168.20.2,53,17,5/2/2020 13:35,3225,1,3,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NORMAL
2,176020,28.240.238.46,0,192.168.3.130,0,0,10/1/2020 5:56,44,0,2,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDOS
3,119202,255.79.185.131,0,192.168.20.134,0,0,4/2/2020 15:20,17,0,2,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDOS
4,80743,192.168.20.133,49140,74.125.193.157,443,6,5/2/2020 13:43,29567478,4,8,...,0,16078.0,0.0,16078.0,16078.0,29500000.0,0.0,29500000.0,29500000.0,NORMAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275105,61870,192.168.20.133,49709,192.168.20.2,53,17,5/2/2020 12:07,3287,1,3,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NORMAL
275106,151697,200.175.2.130,43869,192.168.3.130,6378,6,9/1/2020 17:32,11,0,2,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,PROBE
275107,122318,39.8.159.32,0,192.168.20.134,0,0,4/2/2020 15:20,22,0,2,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDOS
275108,162850,192.168.3.130,80,200.175.2.130,43989,6,9/1/2020 16:30,62399408,3,7,...,0,1317.0,0.0,1317.0,1317.0,62400000.0,0.0,62400000.0,62400000.0,PROBE


In [12]:

# Fill missing values
df_train.fillna("missing", inplace=True)
df_test.fillna("missing", inplace=True)

# Identify label column
label_col = [col for col in df_train.columns if "label" in col.lower()][0]

# Separate features and target
X_train = df_train.drop(columns=[label_col])
X_test = df_test.drop(columns=[label_col])
y_train = df_train[label_col]
y_test = df_test[label_col]

# Ensure both sets have the same columns in the same order
X_test = X_test[X_train.columns]

# Encode all object columns (string/categorical) safely
for col in X_train.columns:
    if X_train[col].dtype == "object" or X_train[col].dtype.name == "category":
        # Fit encoder on both train+test values to avoid unseen error
        all_vals = pd.concat([X_train[col], X_test[col]]).astype(str).unique()
        encoder = LabelEncoder().fit(all_vals)

        X_train[col] = encoder.transform(X_train[col].astype(str))
        X_test[col] = encoder.transform(X_test[col].astype(str))

# Encode labels (shared)
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Train and evaluate classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

# Output metrics
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


ValueError: could not convert string to float: '192.168.20.134-251.238.13.47-0-0-0'